In [1]:
# initial install
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# install packages
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
# update packages
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
# import packages + load dataframe
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv('data.csv')


/var/folders/v1/rpt43fxj323_cj0c2q98_b7r0000gn/T/ipykernel_31202/4076321326.py:5: DtypeWarning: Columns (690,691,692,703,706,708,709,710,711,712,713,714,715,716,717,718,719,720,721,1172,1174,1185,1214) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


# Data Exploration

In [5]:
# analyze dataframe

# print(list(df.columns))

In [6]:
# View ranges of data in each column

"""
for col in df.columns:
    unique_values = df[col].dropna().unique()  # Drop NaNs if you don’t want them in the list
    unique_values_list = [float(val) if isinstance(val, (float, int)) else val for val in unique_values]
    print(f"{col}: {unique_values_list}")
"""


'\nfor col in df.columns:\n    unique_values = df[col].dropna().unique()  # Drop NaNs if you don’t want them in the list\n    unique_values_list = [float(val) if isinstance(val, (float, int)) else val for val in unique_values]\n    print(f"{col}: {unique_values_list}")\n'

# Clean Dataframe

In [7]:
# 1. Drop rows that aren't questions, or questions that have text values.
columns_to_keep = [col for col in df.columns if ('Q' in col or 'QN' in col) and 'TEXT' not in col]
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]
df = df[columns_to_keep]

# Print dropped columns
# print("Dropped columns:", columns_to_drop)

In [8]:
# 2. Drop all QN columns

columns_to_drop = [col for col in df.columns if col.startswith("QN")]
columns_to_keep = [col for col in df.columns if col.startswith("Q") and not col.startswith("QN")]
df_old = df[columns_to_drop]
df = df[columns_to_keep]

# print("Dropped columns:", columns_to_drop)
# print("\n")
# print("Kept columns:", columns_to_keep)


In [9]:
# 3. Merge all multi-choice questions into one column.

In [10]:
# 3: Question 4a

# Debug print to display the value counts of Q4a before transformations
print("Initial value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))
print("Initial value counts in Q4b:")
print(df['Q4b'].value_counts(dropna=False))

# Transformation code
main_column = 'Q4a'
additional_columns = ['Q4b', 'Q4c', 'Q4d', 'Q4e']

for index, row in df.iterrows():
    for col in additional_columns:
        try:
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1]  # Use suffix (b, c, d, or e)
                break
        except (ValueError, TypeError):
            continue

df = df.drop(columns=additional_columns)

# Display the updated value counts in Q4a after transformations
print("Updated value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))

Initial value counts in Q4a:
Q4a
1      15287
NaN     6266
N        492
Z         24
Name: count, dtype: int64
Initial value counts in Q4b:
Q4b
NaN    17689
1       3864
N        492
Z         24
Name: count, dtype: int64
Updated value counts in Q4a:
Q4a
1    15287
b     3864
e     1863
N      492
c      418
d      121
Z       24
Name: count, dtype: int64


In [11]:
# 3: Question 5

# Debug print to display the value counts of QN5A before transformations
print("Initial value counts in Q5a:")
print(df['Q5a'].value_counts(dropna=False))

# Define the main column and additional columns to check for Q5
main_column = 'Q5'
additional_columns = ['Q5a', 'Q5b', 'Q5c', 'Q5d', 'Q5e']

# Initialize Q5 column with values from Q5a where appropriate
df[main_column] = df['Q5a']

# Transformation logic for Q5
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1]  # Set Q5 to the suffix (a, b, c, d, or e)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q5a to Q5e
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop the additional columns
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q5 after transformations
print("Updated value counts in Q5:")
print(df['Q5'].value_counts(dropna=False))


Initial value counts in Q5a:
Q5a
NaN    17990
1       2438
N       1588
Z         53
Name: count, dtype: int64
Updated value counts in Q5:
Q5
e    10949
c     3854
b     2750
a     2438
N     1588
d      437
Z       53
Name: count, dtype: int64


In [12]:
# 3: Question 11

# Debug print to display the initial value counts in Q11a before transformation
print("Initial value counts in Q11a:")
print(df['Q11a'].value_counts(dropna=False))

# Define the main column and all Q11* columns to check for Q11
main_column = 'Q11'
additional_columns = ['Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'Q11f', 'Q11g', 'Q11h', 'Q11i', 'Q11j', 'Q11k', 'Q11l', 'Q11m', 'Q11n']

# Initialize Q11 column with values from Q11a where appropriate
df[main_column] = df['Q11a']

# Transformation logic for Q11
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1].lower()  # Set Q11 to the suffix in lowercase (a, b, c, etc.)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q11a to Q11n
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop all Q11* columns, leaving only Q11
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q11 after transformations
print("Updated value counts in Q11:")
print(df['Q11'].value_counts(dropna=False))


Initial value counts in Q11a:
Q11a
S      18518
1       1654
NaN     1631
Z        213
N         42
1.0       11
Name: count, dtype: int64
Updated value counts in Q11:
Q11
S    18518
a     1665
k      479
b      416
Z      213
l      191
n      141
m       73
j       73
h       70
N       42
i       36
c       34
f       33
g       33
e       32
d       20
Name: count, dtype: int64


In [13]:
# Question 12

# Debug print to display the initial value counts in Q12a before transformation
print("Initial value counts in Q12a:")
print(df['Q12a'].value_counts(dropna=False))

# Define the main column and all Q12* columns to check for Q12
main_column = 'Q12'
additional_columns = ['Q12a', 'Q12b', 'Q12c', 'Q12d', 'Q12e', 'Q12f', 'Q12g', 'Q12h', 'Q12i', 'Q12j', 'Q12k', 'Q12l', 'Q12m', 'Q12n']

# Initialize Q12 with values from Q12a where appropriate
df[main_column] = df['Q12a']

# Transformation logic for Q12
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1].lower()  # Set Q12 to the suffix in lowercase (a, b, c, etc.)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q12a to Q12n
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop all Q12* columns, leaving only Q12
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q12 after transformations
print("Updated value counts in Q12:")
print(df['Q12'].value_counts(dropna=False))

Initial value counts in Q12a:
Q12a
S      20301
NaN     1043
1        430
Z        219
N         76
Name: count, dtype: int64
Updated value counts in Q12:
Q12
S    20301
a      430
l      291
Z      219
n      149
m      129
j       76
N       76
k       68
h       67
b       64
g       47
c       40
i       39
e       36
d       28
f        9
Name: count, dtype: int64
